# 目的
あるクラウドファンディングが成功するか(state)を事前に予測するモデルを構築する。

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, SGDClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error # 回帰問題における性能評価に関する関数
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix # 回帰問題における性能評価に関する関数

# データの読み込み

## データの扱いについて
| Variables | 変数 | 使う? | 変数の扱い |
| ---- | ---- | ---- |  ---- |
| ID | internal kickstarter id | - | - |
| name | プロジェクト名 | - | - |
| category | カテゴリー | ○ | ダミー変数 |
| main_category | category of campaign | ○ | ダミー変数 |
| currency | 通貨 | ○ | ダミー変数 |
| deadline | 期限 | ○ (開始日 - 期限 の募集機関で使ってみる) | 数値(日数) |
| goal | 目標金額 | ○ | 数値(金額) |
| launched | 開始日 | ○ (開始日 - 期限 の募集機関で使ってみる) | 数値(日数) |
| pledged | 集まった金額 | - (開始時にはわからないので使わない) | - |
| state | 成否 | ※目的変数 | fiailled: 0, successful: 1, Other: 捨てる(Pandasで) |

↓必要な項目を抜き出して読み込み、データの整形を行う。

In [64]:
ks_projects_data = pd.read_csv("ks-projects-201801.csv")[['category', 'main_category', 'currency', 'deadline', 'goal', 'launched', 'state']]

# state が failed と successful 以外の行を削除 
# SettingWithCopyWarningがでるので copy() する
# indexが連番じゃなくなるので reset_index() で連番に直す
ks_projects_data2 = ks_projects_data[(ks_projects_data['state'] == 'failed') | (ks_projects_data['state'] == 'successful')].copy().reset_index(drop=True)

# 欠損値が無いことを確認
ks_projects_data2.info()

# deadline - launched で募集期間を算出する
# pd.to_datetime() を使って、str を datetime64型に変換
# 日数(int)で出したいので、Series.dt() を使って変換
ks_projects_data2['launched'] = pd.to_datetime(ks_projects_data2['launched'], format='%Y-%m-%d %H:%M:%S')
ks_projects_data2['deadline'] = pd.to_datetime(ks_projects_data2['deadline'], format='%Y-%m-%d %H:%M:%S')
ks_projects_data2['duration'] = (ks_projects_data2['deadline'] - ks_projects_data2['launched']).dt.days

display(ks_projects_data2.head())

# currency を



#launched_day = datetime.datetime.strptime(ks_projects_data2.loc[0, 'launched'], '%Y-%m-%d %H:%M:%S')
#deadline_day = datetime.datetime.strptime(ks_projects_data2.loc[0, 'deadline'], '%Y-%m-%d')
#duration = deadline_day - launched_day
#print(duration.days)


#print(type(datetime.datetime.strptime(ks_projects_data2.loc[1, 'launched'], '%Y-%m-%d %H:%M:%S')))




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331675 entries, 0 to 331674
Data columns (total 7 columns):
category         331675 non-null object
main_category    331675 non-null object
currency         331675 non-null object
deadline         331675 non-null object
goal             331675 non-null float64
launched         331675 non-null object
state            331675 non-null object
dtypes: float64(1), object(6)
memory usage: 17.7+ MB


,category,main_category,currency,deadline,goal,launched,state,duration
0,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,failed,58
1,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,failed,59
2,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,failed,44
3,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,failed,29
4,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,successful,34


,category,main_category,currency,deadline,goal,launched,state,duration
0,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,failed,58
1,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,failed,59
2,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,failed,44
3,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,failed,29
4,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,successful,34
